In [ ]:
from skimage.metrics import structural_similarity as ssim
import cv2
import

In [ ]:


def rerank_ssim(query_img, candidate_imgs, faiss_indices, faiss_scores):
    """
    Re-rank les résultats FAISS à l’aide de SSIM.
    
    Args:
        query_img: image numpy RGB
        candidate_imgs: liste d’images numpy RGB
        faiss_indices: indices retournés par FAISS
        faiss_scores: scores FAISS (cosine/IP)
        
    Returns:
        reranked_indices: indices triés selon la SSIM
    """
    query_gray = cv2.cvtColor(query_img, cv2.COLOR_RGB2GRAY)
    ssim_scores = []
    
    for img in candidate_imgs:
        cand_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        s, _ = ssim(query_gray, cand_gray, full=True)
        ssim_scores.append(s)
    
    # Fusionner FAISS + SSIM (pondération optionnelle)
    combined = np.array(ssim_scores) * 0.7 + faiss_scores * 0.3
    sorted_idx = np.argsort(-combined)
    
    return [faiss_indices[i] for i in sorted_idx]